In [20]:


#@title The MIT License (MIT)
#
# Copyright (c) 2024 Eric dos Santos.
#
# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

# Fake News Classification System

This project aims to develop a neural network for detecting fake news in Portuguese, using the dataset [Fake.br-Corpus](https://github.com/roneysco/Fake.br-Corpus). With this, we seek to create a system capable of identifying patterns and distinguishing fake news from real news, contributing to the fight against misinformation.

<table class="tfo-notebook-buttons" align="center">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ericshantos/br_fake_news_detector_model/blob/main/br_fake_news_detector_model.ipynb
"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/ericshantos/br_fake_news_detector_model/blob/main/br_fake_news_detector_model/br_fake_news_detector_model.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View the code on GitHub</a>
  </td>
</table>

## Dataset loading

In [21]:
!git clone https://github.com/roneysco/Fake.br-Corpus
DATA_PATH = "./Fake.br-Corpus/full_texts"

fatal: destination path 'Fake.br-Corpus' already exists and is not an empty directory.


In [22]:
import pandas as pd
import os

# News Directory
fake_dir = f"{DATA_PATH}/fake"
real_dir = f"{DATA_PATH}/true"

### News content extraction:


In [23]:
import os
import pandas as pd

def load_news(news_dir: str, label: str) -> pd.DataFrame:
    # List to store news
    news = []

    # Cycle through all files in the specified directory
    for filename in os.listdir(news_dir):
        # Checks if the file has the .txt extension
        if filename.endswith(".txt"):
            # Gets the full path of the file
            file_path = os.path.join(news_dir, filename)

            # Open the file and read its contents
            with open(file_path, "r") as file:
                content = file.read()

                # Adds the content and label to the news list
                news.append({"text": content, "label": label})

    # Returns a pandas DataFrame containing the news
    return pd.DataFrame(news)

Result:

In [24]:
fake_news = load_news(fake_dir, 0)
real_news = load_news(real_dir, 1)

## Data preprocessing

### Concatenate the DataFrames

Group Dataframes to generate a single robust database.

In [25]:
data_news = pd.concat([fake_news, real_news], ignore_index=True).sample(frac=1, random_state=13)

Final base information:

In [26]:
data_news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7200 entries, 3248 to 338
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7200 non-null   object
 1   label   7200 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 168.8+ KB


In [27]:
data_news = data_news.apply(

    # If valid, type the column as float
    lambda col: col.astype(float) if col.apply(

        # Check if they are digits
        lambda x: str(x).replace('.', '', 1).isdigit()
    ).all() else col
)

# Result
print(data_news.dtypes)

text      object
label    float64
dtype: object


### Data cleaning

In [28]:
!python -m spacy download pt_core_news_sm > /dev/null 2>&1
!pip install unidecode > /dev/null 2>&1

from unidecode import unidecode
import spacy

nlp = spacy.load("pt_core_news_sm")

def clean_text(text):

  # Text processing
  doc = nlp(text)

  # Tokenization, stopword removal, punctuation and accentuation
  tokens = [unidecode(token.lemma_) for token in doc if not token.is_stop and not token.is_punct]

  return ' '.join(tokens)

Clear news content:

In [29]:
data_news["text"] = data_news["text"].apply(clean_text)

In [30]:
data_news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7200 entries, 3248 to 338
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   text    7200 non-null   object 
 1   label   7200 non-null   float64
dtypes: float64(1), object(1)
memory usage: 168.8+ KB


## Training

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

# Tokenizer Object
tokenizer = Tokenizer(num_words=10000)

tokenizer.fit_on_texts(data_news['text'])

# Converting texts into sequences of numbers
sequences = tokenizer.texts_to_sequences(data_news['text'])

### Prepares the labels and data for training

In [32]:
# Transform the texts into sequences of numbers
X = pad_sequences(sequences, maxlen=200)

# news labels(fake or real)
y = data_news["label"]

### Splitting the dataset into training and testing

In [33]:
from sklearn.model_selection import train_test_split

# Splits data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")

Training set size: (5760, 200)
Test set size: (1440, 200)


### Model architecture

In [34]:
model = Sequential([
  # Convert tokens to dense vectors: input layer
  Embedding(input_dim=10000, output_dim=128, input_length=200),

  # Hidden layers
  LSTM(128, return_sequences=True),
  Dropout(0.2),
  LSTM(64, return_sequences=True),
  Dropout(0.2),
  LSTM(32),

  # Output Layer
  Dense(1, activation="sigmoid")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


**Model compilation**:

In [35]:
model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)

### Treinando o modelo

In [36]:
history = model.fit(X_train, y_train, epochs=5, batch_size=128, validation_data=(X_test, y_test))

Epoch 1/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.7398 - loss: 0.4823 - val_accuracy: 0.9389 - val_loss: 0.2268
Epoch 2/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.9444 - loss: 0.2152 - val_accuracy: 0.9389 - val_loss: 0.2260
Epoch 3/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 65s 1s/step - accuracy: 0.9441 - loss: 0.2159 - val_accuracy: 0.9389 - val_loss: 0.2270
Epoch 4/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.9439 - loss: 0.2152 - val_accuracy: 0.9389 - val_loss: 0.2297
Epoch 5/5
45/45 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.9481 - loss: 0.2044 - val_accuracy: 0.9389 - val_loss: 0.2298


#### Model evaluation

In [37]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")

45/45 ━━━━━━━━━━━━━━━━━━━━ 7s 160ms/step - accuracy: 0.9252 - loss: 0.2690
Loss: 0.22978998720645905, Accuracy: 0.9388889074325562


### Save the model

In [38]:
model.save("br_fake_news_predict_model.keras")